# Farmers using the Branch-and-Bound on Dual Decomposition

This note has been created by using Julia 1.0.1 and DSP (branch `dsp-bb`).

## Required Packages

In [1]:
using JuMP, Dsp

┌ Info: Precompiling Dsp [3f8b1f33-babd-5f87-a419-0f6725e165cf]
└ @ Base loading.jl:1189
┌ Warning: Package Dsp does not have MPI in its dependencies:
│ - If you have Dsp checked out for development and have
│   added MPI as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Dsp
│ Loading MPI into Dsp from project dependency, future warnings for Dsp are suppressed.
└ @ nothing nothing:837


## Parameters

In [2]:
NS = 3;                        # number of scenarios
probability = [1/3, 1/3, 1/3]; # probability

CROPS  = 1:3 # set of crops (wheat, corn and sugar beets, resp.)
PURCH  = 1:2 # set of crops to purchase (wheat and corn, resp.)
SELL   = 1:4 # set of crops to sell (wheat, corn, sugar beets under 6K and those over 6K)

Cost     = [150 230 260]   # cost of planting crops
Budget   = 500             # budget capacity
Purchase = [238 210];      # purchase price
Sell     = [170 150 36 10] # selling price
Yield    = [3.0 3.6 24.0;
            2.5 3.0 20.0;
            2.0 2.4 16.0]
Minreq   = [200 240 0]     # minimum crop requirement

1×3 Array{Int64,2}:
 200  240  0

## JuMP Model

Note that the JuMP model is **different** from the one in `Farmers-DD` notebook. In particular, we need to define all the variables and the objective function in the master. Some of the variables are splitted into subproblems, whereas some are coupled in the master.

In [3]:
m = Model(NS)

@variable(m, x[i=CROPS,s=1:NS] >= 0, Int)
@variable(m, y[j=PURCH,s=1:NS] >= 0)
@variable(m, w[k=SELL,s=1:NS] >= 0)
@objective(m, Min, 
	  sum(probability[s] * Cost[i] * x[i,s] for i=CROPS for s=1:NS)
    + sum(probability[s] * Purchase[j] * y[j,s] for j=PURCH for s=1:NS) 
	- sum(probability[s] * Sell[k] * w[k,s] for k=SELL for s=1:NS))
@constraint(m, nonant1[i=CROPS,s=1], x[i,NS] - x[i,1] == 0)
@constraint(m, nonant2[i=CROPS,s=2:NS], x[i,s-1] - x[i,s] == 0)

for s in 1:NS
    blk = Model(m, s, 1.0)
	@objective(blk, Min, 0)
	@constraint(blk, const_budget, sum(x[i,s] for i=CROPS) <= Budget)
    @constraint(blk, const_minreq[j=PURCH], Yield[s,j] * x[j,s] + y[j,s] - w[j,s] >= Minreq[j])
    @constraint(blk, const_minreq_beets, Yield[s,3] * x[3,s] - w[3,s] - w[4,s] >= Minreq[3])
    @constraint(blk, const_aux, w[3,s] <= 6000)
end

### Query the master problem

In [4]:
print(m)

Min 50 x[1,1] + 50 x[1,2] + 50 x[1,3] + 76.66666666666666 x[2,1] + 76.66666666666666 x[2,2] + 76.66666666666666 x[2,3] + 86.66666666666666 x[3,1] + 86.66666666666666 x[3,2] + 86.66666666666666 x[3,3] + 79.33333333333333 y[1,1] + 79.33333333333333 y[1,2] + 79.33333333333333 y[1,3] + 70 y[2,1] + 70 y[2,2] + 70 y[2,3] - 56.666666666666664 w[1,1] - 56.666666666666664 w[1,2] - 56.666666666666664 w[1,3] - 50 w[2,1] - 50 w[2,2] - 50 w[2,3] - 12 w[3,1] - 12 w[3,2] - 12 w[3,3] - 3.333333333333333 w[4,1] - 3.333333333333333 w[4,2] - 3.333333333333333 w[4,3]
Subject to
 x[1,3] - x[1,1] = 0
 x[2,3] - x[2,1] = 0
 x[3,3] - x[3,1] = 0
 x[1,1] - x[1,2] = 0
 x[1,2] - x[1,3] = 0
 x[2,1] - x[2,2] = 0
 x[2,2] - x[2,3] = 0
 x[3,1] - x[3,2] = 0
 x[3,2] - x[3,3] = 0
 x[i,s] ≥ 0, integer, ∀ i ∈ {1,2,3}, s ∈ {1,2,3}
 y[j,s] ≥ 0 ∀ j ∈ {1,2}, s ∈ {1,2,3}
 w[k,s] ≥ 0 ∀ k ∈ {1,2,3,4}, s ∈ {1,2,3}


### Query a subproblem

In [5]:
print(m.ext[:DspBlocks].children[1])

Min 0
Subject to
 x[1,1] + x[2,1] + x[3,1] ≤ 500
 3 x[1,1] + y[1,1] - w[1,1] ≥ 200
 3.6 x[2,1] + y[2,1] - w[2,1] ≥ 240
 24 x[3,1] - w[3,1] - w[4,1] ≥ 0
 w[3,1] ≤ 6000


## Branch-and-Bound Solution

In [6]:
solve(m, solve_type=:BB)

Initializing subproblems ... 
Initializing master problem ... 
Initializing ALPS framework ... 

*** WARNING: Changed the branching method. The problem structure supports the integer branching method only. ***

==  Welcome to the Abstract Library for Parallel Search (ALPS) 
==  Copyright 2000-2017 Lehigh University and others 
==  All Rights Reserved. 
==  Distributed under the Eclipse Public License 1.0 
==  Version: Trunk (unstable) 
==  Build Date: Jul 23 2018
Alps0250I Starting search ...
Generated 3 initial columns. Initial dual bound -1.154000000000e+05
Iteration   0: DW Bound +1.797693e+308, Best Dual -1.154000e+05 (gap Large %), nrows 3, ncols 12, timing (total 0.00, master 0.00, gencols 0.00), statue 3000
Iteration   1: DW Bound +1.797693e+308, Best Dual -1.154000e+05 (gap Large %), nrows 6, ncols 12, timing (total 0.01, master 0.00, gencols 0.00), statue 3000
Iteration   2: DW Bound -1.052861e+05, Best Dual -1.154000e+05 (gap 8.76 %), nrows 9, ncols 12, timing (total 0.01, ma

:Optimal

## Primal and Dual Bounds

This aims to find a **global optimal** solution.

In [7]:
(getprimobjval(), getdualobjval())

(-108390.00000000001, -108390.00000000001)

## Primal Solution

In [8]:
getprimvalue()

27-element Array{Float64,1}:
  170.0             
  170.0             
  170.0             
   80.0             
   80.0             
   80.0             
  250.0             
  249.99999999999997
  249.99999999999994
    0.0             
    0.0             
    0.0             
    0.0             
    ⋮               
  310.0             
  225.00000000000006
  140.00000000000003
   48.00000000000005
    0.0             
    0.0             
 6000.0             
 5000.0             
 3999.999999999999  
    0.0             
    0.0             
    0.0             

## Dual Solution

There is no meaningful dual solution to report.